## WEEK 3. Построение baseline-решений

В этом задании вам предстоит построить несколько моделей и оценить их качество. Эти модели будут служить нам в качестве baseline-решений и пригодятся сразу для нескольких задач:

1. Во-первых, на разработку baseline-модели не должно уходить много времени (это требование исходит из оценок затрат на проект в целом - большую часть времени все же нужно потратить на основное решение), процесс должен быть простым, на подавляющем большинстве этапов должны использоваться готовые протестированные инструменты.  Все это приводит к тому, что baseline-модели - это дешевый способ сделать грубую оценку потенциально возможного качества модели, при построении которого вероятность допущения ошибок относительно невелика.
2. Во-вторых, использование моделей разного типа при построении baseline'ов позволяет на раннем этапе сделать предположения о том, какие подходы являются наиболее перспективными и приоритизировать дальнейшие эксперименты.
3. Наличие baseline-моделей позволяет оценить, какой прирост качества дают различные преобразования, усложнения, оптимизации и прочие активности, которые вы предпринимаете для построения финального решения.
4. Наконец, если после построение сложного решения оценка его качества будет очень сильно отличаться от оценки качества baseline-моделей, то это будет хорошим поводом поискать в решении ошибки.

## Задание

Обучите 3 разные baseline-модели на полученных наборах данных и оцените их качество. На прошлой неделе вы выбрали методику оценки качества моделей на основе кросс-валидации, а также основную и вспомогательные метрики. Оцените с их помощью получившуюся модель. Обратите внимание, что под разными моделями понимаются именно разные алгоритмы классификации. Например, 2 модели, реализующие метод k ближайших соседей с разными k, будут считаться одним baseline-решением (хотя и с разными параметрами). Напоминаем, что отложенная выборка (hold-out dataset) не должна использоваться для построения baseline-моделей!

In [7]:
import pandas as pd
import numpy as np
import collections
import scipy
from scipy import stats
import seaborn as sns
import random
from matplotlib import pyplot as plt
from sklearn import model_selection, linear_model, metrics, ensemble
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## I. Подготовка данных

In [8]:
X = pd.read_csv('orange_small_churn_data.txt', header=0, sep=',')
y = pd.read_csv('orange_small_churn_labels.txt', header=None, names=['Labels'], sep=',')

#Создаем общую таблицу
data = pd.DataFrame(np.hstack((X,y)), columns = list(X.columns) + list(y.columns))
data.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,Labels
0,NaN,NaN,NaN,NaN,NaN,3052,NaN,NaN,NaN,NaN,...,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,-1
1,NaN,NaN,NaN,NaN,NaN,1813,7,NaN,NaN,NaN,...,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN,-1
2,NaN,NaN,NaN,NaN,NaN,1953,7,NaN,NaN,NaN,...,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN,-1
3,NaN,NaN,NaN,NaN,NaN,1533,7,NaN,NaN,NaN,...,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,1
4,NaN,NaN,NaN,NaN,NaN,686,7,NaN,NaN,NaN,...,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,-1


### Удаление пустых столбцов и обработка пропусков

In [9]:
X_del = X.iloc[:, 0:190].dropna(axis = 1, thresh=1)  #удаляем пустые столбцы у числовых признаков (останется 174 признака)
X_zero = X_del.fillna(0)                             #заполняем пропуски нулями
X_mean = X_del.fillna(X_del.mean())                  #заполняем пропуски средним значением по столбцу

In [10]:
#Проверим есть ли строка "Na" среди знач-й категор.приз-ов. Если нет, то заполним пропуски словом "Na"
print ('Na' in X.iloc[:, 190:].values)

False


In [11]:
X_cat = X.iloc[:, 190:].dropna(axis = 1, thresh=1)     #удаляем пустые столбцы у категориальных признаков (останется 38 приз-ов)
X_cat=X_cat.fillna('Na', axis=0).applymap(str)         #приводим все значения к строковым и заполняем пропуски строкой "Na"

### Обработка категориальных признаков

Заменим каждую категорию числом входящих в неё объектов. Код: data[newfeature]=data[feature].map(data.groupby(feature).size()).

In [12]:
for j in range(X_cat.shape[1]):
    X_cat.iloc[:,j] = X_cat.iloc[:,j].map(X_cat.groupby(X_cat.columns[j]).size())
X_cat.head(3)

,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,39129,222,1763,29810,38353,39633,96,3,46,20366,...,3,1344,3,29279,39338,20935,2117,1896,1193,22777
1,39129,298,5781,29810,38353,39633,273,1,240,1,...,1,29610,1,29279,39338,8875,1692,28112,3457,7850
2,39129,274,349,29810,38353,39633,3334,3557,119,20366,...,3557,4960,3557,29279,39338,20935,6403,4928,2111,7850


In [13]:
#Объединяем вещ. и катег. пер-ые, у вещ.пер-х пропуски заполнены нулями
X_zero = pd.DataFrame(np.hstack((X_zero, X_cat)), columns = list(X_zero.columns)+list(X_cat.columns))
X_zero.head(3)

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0.0,0.0,0.0,0.0,0.0,3052.0,0.0,0.0,0.0,0.0,...,3.0,1344.0,3.0,29279.0,39338.0,20935.0,2117.0,1896.0,1193.0,22777.0
1,0.0,0.0,0.0,0.0,0.0,1813.0,7.0,0.0,0.0,0.0,...,1.0,29610.0,1.0,29279.0,39338.0,8875.0,1692.0,28112.0,3457.0,7850.0
2,0.0,0.0,0.0,0.0,0.0,1953.0,7.0,0.0,0.0,0.0,...,3557.0,4960.0,3557.0,29279.0,39338.0,20935.0,6403.0,4928.0,2111.0,7850.0


In [14]:
#Объединяем вещ. и катег. пер-ые, у вещ.пер-х пропуски заполнены средними значениями
X_mean = pd.DataFrame(np.hstack((X_mean, X_cat)), columns = list(X_mean.columns)+list(X_cat.columns))
X_mean.head(3)

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,11.003509,0.00497,429.328358,0.148235,249055.515021,3052.0,6.792126,47.487719,381346.56309,8.549254,...,3.0,1344.0,3.0,29279.0,39338.0,20935.0,2117.0,1896.0,1193.0,22777.0
1,11.003509,0.00497,429.328358,0.148235,249055.515021,1813.0,7.000000,47.487719,381346.56309,8.549254,...,1.0,29610.0,1.0,29279.0,39338.0,8875.0,1692.0,28112.0,3457.0,7850.0
2,11.003509,0.00497,429.328358,0.148235,249055.515021,1953.0,7.000000,47.487719,381346.56309,8.549254,...,3557.0,4960.0,3557.0,29279.0,39338.0,20935.0,6403.0,4928.0,2111.0,7850.0


### Отделяем hold-out dataset с сохранением баланса классов (стратификация)

In [15]:
# X_test_zero, y_test_zero, X_test_mean, y_test_mean - это наши hold-out dataset.
from sklearn.model_selection import train_test_split

X_train_zero, X_test_zero, y_train_zero, y_test_zero = train_test_split(X_zero, y, test_size=0.3, stratify=y, random_state=0)
X_train_mean, X_test_mean, y_train_mean, y_test_mean = train_test_split(X_mean, y, test_size=0.3, stratify=y, random_state=0)

### Балансировка классов

In [16]:
# покажем, что классы в выборке не сбалансированы

print(np.sum(y_train_zero.Labels==1))
print(np.sum(y_train_zero.Labels==-1))

2083
25917


In [17]:
print(np.sum(y_train_mean.Labels==1))
print(np.sum(y_train_mean.Labels==-1))

2083
25917


In [18]:
# Добавление данных класса 1 в обучающую выборку

np.random.seed(0)
indices_to_add = np.random.randint(np.sum(y_train_zero.Labels==1),
                                   size=(np.sum(y_train_zero.Labels==-1)-np.sum(y_train_zero.Labels==1)))

X_train_zero_add = X_train_zero.to_numpy()[y_train_zero.Labels == 1,:][indices_to_add,:]
X_train_zero = np.vstack((X_train_zero, X_train_zero_add))

y_train_zero_add = y_train_zero.to_numpy()[y_train_zero.Labels == 1][indices_to_add]
y_train_zero = np.vstack((y_train_zero, y_train_zero_add))

In [19]:
np.random.seed(0)
indices_to_add = np.random.randint(np.sum(y_train_mean.Labels==1),
                                   size=(np.sum(y_train_mean.Labels==-1)-np.sum(y_train_mean.Labels==1)))

X_train_mean_add = X_train_mean.to_numpy()[y_train_mean.Labels == 1,:][indices_to_add,:]
X_train_mean = np.vstack((X_train_mean, X_train_mean_add))

y_train_mean_add = y_train_mean.to_numpy()[y_train_mean.Labels == 1][indices_to_add]
y_train_mean = np.vstack((y_train_mean, y_train_mean_add))

## II. Построение baseline моделей

### 1. LogisticRegression

#### Линейная модель LogisticRegression с регуляризацией Лассо (для отбора признаков, т.к. у нас их много и много пропусков)

In [20]:
# Создаем регрессор
lr_classifier = linear_model.LogisticRegression(solver='liblinear', penalty='l1', random_state = 0)

In [21]:
# Создаем скалер для масштабирования обучающей выборки. StandardScaler отнимает среднее и делит на выборочное стандарт.откл.
scaler = StandardScaler()

In [22]:
# Создаем pipeline из двух шагов: scaling и классификация
lr_pipeline = Pipeline(steps = [('scaling', scaler), ('regression', lr_classifier)])

In [23]:
# Для кросс-валидации будем использовать стратегию StratifiedKFold, делим на 3 фолда, перемешиваем выборку
cv_strategy = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

#### Создадим функцию для вычисления метрик качества

In [24]:
def metric(pipeline, X, y, cv):
    recall = model_selection.cross_val_score(pipeline, X, y, scoring = 'recall', cv=cv)
    precision = model_selection.cross_val_score(pipeline, X, y, scoring = 'precision', cv=cv)
    F1 = model_selection.cross_val_score(pipeline, X, y, scoring = 'f1', cv=cv)
    ROC_AUC = model_selection.cross_val_score(pipeline, X, y, scoring = 'roc_auc', cv=cv)
    return print("Recall: %s,\nPrecision: %r,\nF1: %f,\nROC-AUC: %g." %(recall.mean(), precision.mean(), F1.mean(), ROC_AUC.mean()))

In [25]:
print ('Качество LogisticRegression, пропуски в данных заполнены нулями:\n')
metric(lr_pipeline, X_train_zero, y_train_zero.ravel(), cv_strategy)

Качество LogisticRegression, пропуски в данных заполнены нулями:

Recall: 0.672415788864452,
Precision: 0.6345854642762175,
F1: 0.652947,
ROC-AUC: 0.69606.


In [26]:
print ('Качество LogisticRegression, пропуски в данных заполнены средними значениями:\n')
metric(lr_pipeline, X_train_mean, y_train_mean.ravel(), cv_strategy)

Качество LogisticRegression, пропуски в данных заполнены средними значениями:

Recall: 0.6614577304471968,
Precision: 0.6359433427570091,
F1: 0.648432,
ROC-AUC: 0.701643.


### 2. RandomForestClassifier

In [30]:
rf_classifier = ensemble.RandomForestClassifier(n_estimators=10, max_features='sqrt', random_state = 1)
rf_pipeline = Pipeline(steps = [('scaling', scaler), ('regression', rf_classifier)])
cv_strategy = model_selection.StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)

In [31]:
print ('Качество RandomForestClassifier, пропуски в данных заполнены нулями:\n')
metric(rf_pipeline, X_train_zero, y_train_zero.ravel(), cv_strategy)

Качество RandomForestClassifier, пропуски в данных заполнены нулями:

Recall: 1.0,
Precision: 0.994323908729589,
F1: 0.997153,
ROC-AUC: 0.999998.


In [30]:
print ('Качество RandomForestClassifier, пропуски в данных заполнены средними значениями:\n')
metric(rf_pipeline, X_train_mean, y_train_mean.ravel(), cv_strategy)

Качество RandomForestClassifier, пропуски в данных заполнены средними значениями:

Recall: 1.0,
Precision: 0.9951256475634797,
F1: 0.997556,
ROC-AUC: 0.999999.


### 3. GradientBoostingClassifier 

In [28]:
gb_classifier = ensemble.GradientBoostingClassifier(n_estimators = 10, max_depth=5, random_state = 1)
gb_pipeline = Pipeline(steps = [('scaling', scaler), ('regression', gb_classifier)])
cv_strategy = model_selection.StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)

In [32]:
print ('Качество GradientBoostingClassifier, пропуски в данных заполнены нулями:\n')
metric(gb_pipeline, X_train_zero, y_train_zero.ravel(), cv_strategy)

Качество GradientBoostingClassifier, пропуски в данных заполнены нулями:

Recall: 0.7473869642682749,
Precision: 0.6779083892723244,
F1: 0.710858,
ROC-AUC: 0.769972.


In [29]:
print ('Качество GradientBoostingClassifier, пропуски в данных заполнены средними значениями:\n')
metric(gb_pipeline, X_train_mean, y_train_mean.ravel(), cv_strategy)

Качество GradientBoostingClassifier, пропуски в данных заполнены средними значениями:

Recall: 0.735849713748413,
Precision: 0.6857789528959584,
F1: 0.709807,
ROC-AUC: 0.773075.


#### Качество RandomForestClassifier кажется подозрительно высоким, скорее всего модель переобучилась. Проверим модели на отложенном датасете.

In [35]:
# LogisticRegression
lr_pipeline.fit(X_train_mean, y_train_mean)
prediction = lr_pipeline.predict(X_test_mean)
metrics.roc_auc_score(y_test_mean, prediction)

0.6398675572671855

In [33]:
# RandomForestClassifier
rf_pipeline.fit(X_train_mean, y_train_mean)
prediction = rf_pipeline.predict(X_test_mean)
metrics.roc_auc_score(y_test_mean, prediction)

0.5024535337873445

In [34]:
# GradientBoostingClassifier
gb_pipeline.fit(X_train_mean, y_train_mean)
prediction = gb_pipeline.predict(X_test_mean)
metrics.roc_auc_score(y_test_mean, prediction)

0.6718325590098796

### Вывод: среди построенных baseline-решений с учетом качества на отложенном датасете лучшее качество показывает  GradientBoostingClassifier